In [104]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import numpy as np

In [80]:
def box_office_1000(url, year):
    col_names=['rank', 'title', 'worldwide $', 'domestic $', 'domestic %', 'foreign $', 'foreign %']
    html_page = requests.get(url)
    soup = BeautifulSoup(html_page.content, 'html.parser')
    container = soup.find('div', class_='a-section imdb-scroll-table-inner')
    tables = container.find('table').findAll('tr')
    for f in range(1,101):
        time.sleep(0.5)
        film_row = tables[f]
        cols = film_row.findAll('td')
        row_dict={}
        row_dict['year']=year
        c=0
        for col in cols:
            col_name = column_names[c]
            row_dict[col_name] = col.text
            c+=1
        box_office_list.append(row_dict)

        

In [81]:
box_office_list = []

def next_year():
    for d in range(10):
        year = 2021 - d
        string_year = str(year)
        base_url = 'https://www.boxofficemojo.com/year/world/'
        url = base_url + string_year + "/"
        box_office_1000(url, string_year)    
        
next_year()
box_office_1000_df = pd.DataFrame(box_office_list)
box_office_1000_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom","$822,009,764",-,-,"$822,009,764",100%
1,2021,2,F9: The Fast Saga,"$714,396,475","$172,929,475",24.2%,"$541,467,000",75.8%
2,2021,3,Detective Chinatown 3,"$686,257,563",-,-,"$686,257,563",100%
3,2021,4,Godzilla vs. Kong,"$467,863,133","$100,563,133",21.5%,"$367,300,000",78.5%
4,2021,5,Black Widow,"$372,750,698","$183,150,698",49.1%,"$189,600,000",50.9%
...,...,...,...,...,...,...,...,...
1005,2012,97,Moonrise Kingdom,"$68,263,166","$45,512,466",66.7%,"$22,750,700",33.3%
1006,2012,98,The Secret of the Gods,"$67,537,798",-,-,"$67,537,798",100%
1007,2012,99,Madea's Witness Protection,"$66,899,242","$65,653,242",98.1%,"$1,246,000",1.9%
1008,2012,100,Beauty and the Beast 2012 3D Release,"$62,767,829","$47,617,067",75.9%,"$15,150,762",24.1%


In [82]:
box_office_1000_df.dtypes

year           object
rank           object
title          object
worldwide $    object
domestic $     object
domestic %     object
foreign $      object
foreign %      object
dtype: object

In [96]:
bo_1000_clean_df = box_office_1000_df.applymap(lambda x: x[1:].replace(',', '') if  x[0] == "$" else x)
bo_1000_clean_df = bo_1000_clean_df.applymap(lambda x: x[:-1] if  x[-1] == "%" else x)
bo_1000_clean_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,"Hi, Mom","822,009,764",-,-,"822,009,764",100
1,2021,2,F9: The Fast Saga,"714,396,475","172,929,475",24.2,"541,467,000",75.8
2,2021,3,Detective Chinatown 3,"686,257,563",-,-,"686,257,563",100
3,2021,4,Godzilla vs. Kong,"467,863,133","100,563,133",21.5,"367,300,000",78.5
4,2021,5,Black Widow,"372,750,698","183,150,698",49.1,"189,600,000",50.9
...,...,...,...,...,...,...,...,...
1005,2012,97,Moonrise Kingdom,"68,263,166","45,512,466",66.7,"22,750,700",33.3
1006,2012,98,The Secret of the Gods,"67,537,798",-,-,"67,537,798",100
1007,2012,99,Madea's Witness Protection,"66,899,242","65,653,242",98.1,"1,246,000",1.9
1008,2012,100,Beauty and the Beast 2012 3D Release,"62,767,829","47,617,067",75.9,"15,150,762",24.1


In [109]:

bo_1000_clean_df[['worldwide $', 'domestic $', 'foreign $']] = bo_1000_clean_df[['worldwide $', 'domestic $', 'foreign $']].astype(float)
bo_1000_clean_df

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,Hi Mom,822009764.0,0.0,0,822009764.0,100
1,2021,2,F9: The Fast Saga,714396475.0,172929475.0,24.2,541467000.0,75.8
2,2021,3,Detective Chinatown 3,686257563.0,0.0,0,686257563.0,100
3,2021,4,Godzilla vs. Kong,467863133.0,100563133.0,21.5,367300000.0,78.5
4,2021,5,Black Widow,372750698.0,183150698.0,49.1,189600000.0,50.9
...,...,...,...,...,...,...,...,...
1005,2012,97,Moonrise Kingdom,68263166.0,45512466.0,66.7,22750700.0,33.3
1006,2012,98,The Secret of the Gods,67537798.0,0.0,0,67537798.0,100
1007,2012,99,Madea's Witness Protection,66899242.0,65653242.0,98.1,1246000.0,1.9
1008,2012,100,Beauty and the Beast 2012 3D Release,62767829.0,47617067.0,75.9,15150762.0,24.1


In [112]:

bo_1000_clean_df.head(1000)

,year,rank,title,worldwide $,domestic $,domestic %,foreign $,foreign %
0,2021,1,Hi Mom,8.220098e+08,0.0,0,8.220098e+08,100
1,2021,2,F9: The Fast Saga,7.143965e+08,172929475.0,24.2,5.414670e+08,75.8
2,2021,3,Detective Chinatown 3,6.862576e+08,0.0,0,6.862576e+08,100
3,2021,4,Godzilla vs. Kong,4.678631e+08,100563133.0,21.5,3.673000e+08,78.5
4,2021,5,Black Widow,3.727507e+08,183150698.0,49.1,1.896000e+08,50.9
5,2021,6,A Quiet Place Part II,2.973723e+08,160072261.0,53.8,1.373000e+08,46.2
6,2021,7,Free Guy,2.771485e+08,102448540.0,37,1.747000e+08,63
7,2021,8,Shang-Chi and the Legend of the Ten Rings,2.621457e+08,150145729.0,57.3,1.120000e+08,42.7
8,2021,9,Cruella,2.222022e+08,86102206.0,38.7,1.361000e+08,61.3
9,2021,10,The Conjuring: The Devil Made Me Do It,2.019651e+08,65565074.0,32.5,1.364000e+08,67.5
